In [1]:
%load_ext lab_black

In [2]:
import re
import json

from uuid import uuid4

from django.db import connections

from rich import print

# Manual Stuff

* Set DELETE_WAGTAIL_IMAGES=False to deactivate post_delete_file_cleanup, to avoid deleting original 🥶
* Start only postgres and then: dropdb homepage && createdb homepage && python manage.py migrate
* Sometimes after convert: python manage.py fixtree
* Sometimes, you have to run: python manage.py sqlsequencereset cast + execute via pgcli

# Create New Empty DB

In [3]:
import os

from django.conf import settings
from django.core.management import call_command

from pathlib import Path

In [4]:
current_working_dir = Path.cwd()
os.chdir(settings.ROOT_DIR)
!dropdb homepage && createdb homepage
call_command("migrate")
os.chdir(current_working_dir)

Operations to perform:
  Apply all migrations: account, admin, auth, authtoken, cast, contenttypes, django_comments, filepond, fluent_comments, indieweb, sessions, sites, socialaccount, taggit, threadedcomments, users, wagtailadmin, wagtailcore, wagtaildocs, wagtailembeds, wagtailforms, wagtailimages, wagtailredirects, wagtailsearch, wagtailusers, watson
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0001_initial... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying users.0001_initial... OK
  Applying account.0001_initial... OK
  Applying accou

# Fetch Legacy Data from Database Restore

In [5]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [dict(zip(columns, row)) for row in cursor.fetchall()]


class Legacy:
    def __init__(self, db_name="legacy"):
        self.db_name = db_name
        self.users = self.fetch_rows("select * from users_user")
        self.blogs = self.fetch_rows("select * from cast_blog")
        self.posts = self.fetch_rows("select * from cast_post")
        self.images = self.fetch_rows("select * from cast_image")
        self.galleries = self.fetch_rows("select * from cast_gallery")
        self.gallery_images = self.fetch_rows("select * from cast_gallery_images")
        self.videos = self.fetch_rows("select * from cast_video")
        self.audios = self.fetch_rows("select * from cast_audio")

    def fetch_rows(self, stmt):
        with connections[self.db_name].cursor() as cursor:
            cursor.execute(stmt)
            rows = dictfetchall(cursor)
        return rows


class Converter:
    def __init__(self, legacy):
        self.legacy = legacy
        self.blog_content_type = ContentType.objects.get(app_label="cast", model="blog")

    def users(self):
        for l_user in self.legacy.users:
            user = User(**l_user)
            user.save()
        return {user.pk: user for user in User.objects.all()}

    def blogs(self):
        blog_legacy_to_wagtail = {}
        root = Page.objects.get(title="Welcome to your new Wagtail site!")
        for l_blog in self.legacy.blogs:
            kwargs = l_blog.copy()
            del kwargs["user_id"]
            del kwargs["id"]
            kwargs["owner"] = self.user_lookup[l_blog["user_id"]]
            l_blog["content_type"] = self.blog_content_type
            blog = Blog(**kwargs)
            blog = root.add_child(instance=blog)
            blog_legacy_to_wagtail[l_blog["id"]] = blog.pk
        return blog_legacy_to_wagtail

    def images(self):
        for num, l_image in enumerate(self.legacy.images):
            image = Image(
                pk=l_image["id"],
                file=l_image["original"],
                uploaded_by_user=self.user_lookup[l_image["user_id"]],
                created_at=l_image["created"],
                width=l_image["original_width"],
                height=l_image["original_height"],
            )
            image.save()
            if num % 300 == 0:
                print(num)

    def galleries(self):
        for l_gallery in self.legacy.galleries:
            kwargs = {k: v for k, v in l_gallery.items() if k != "user_id"}
            gallery = Gallery(**kwargs)
            gallery.save()

    def gallery_image_links(self):
        links = [
            (gi["id"], gi["gallery_id"], gi["image_id"])
            for gi in self.legacy.gallery_images
        ]
        stmt = "insert into cast_gallery_images (id, gallery_id, image_id) values (%s, %s, %s)"
        with connections["default"].cursor() as cursor:
            cursor.executemany(stmt, links)

    def videos(self):
        for num, video in enumerate(self.legacy.videos):
            upload_user = self.user_lookup[video["user_id"]]
            # poster = Image(file=video["poster"], uploaded_by_user=upload_user)
            # poster.save()
            video = Video(
                pk=video["id"],
                title=Path(video["original"]).name,
                user=upload_user,
                poster=video["poster"],
                poster_seconds=video["poster_seconds"],
                original=video["original"],
                created=video["created"],
                modified=video["modified"],
            )
            video.save(poster=False)
            if num % 10 == 0:
                print(num)

    def convert(self):
        self.user_lookup = self.users()
        self.blog_lookup = self.blogs()
        # self.images()
        # self.galleries()
        # self.gallery_image_links()
        self.videos()

In [6]:
converter = Converter(Legacy())

In [7]:
len(converter.legacy.videos)

128

In [8]:
%%time
converter.convert()

0

10

20

30

40

...

90

100

110

120

CPU times: user 1.45 s, sys: 1.16 s, total: 2.61 s
Wall time: 828 ms


In [10]:
video = Video.objects.get(pk=1)
video.original.url

'https://d2b7dn9rofvhjd.cloudfront.net/blogs_videos/testvideo.mp4'

In [18]:
video.poster.url

'https://d2b7dn9rofvhjd.cloudfront.net/blogs_videos/poster/poster_6wrvgvwm.jpg'

In [34]:
#post = Post.objects.first()
#print(post.body._raw_data)

[
    {
        'type': 'overview',
        'value': [
            {'type': 'heading', 'value': 'Eine Überschrift', 'id': '58378935-196d-4396-a23e-2bc1c9cbc0f3'},
            {
                'type': 'paragraph',
                'value': '<p data-block-key="anlsf">Ein bisschen Text. Angeblich geht hier auch RichText. Hmm, wäre
das dann eine Überschrift?</p><p data-block-key="r0vs"></p><p data-block-key="7pj4l"><a 
href="http://wersdoerfer.de">wersdoerfer</a>Aha!</p><p data-block-key="bivd5"></p><h2 
data-block-key="6vmjl">Heading 2</h2>',
                'id': '9279543d-88aa-4be3-a12c-952ebe6966c0'
            }
        ],
        'id': 'bdab0f1c-c164-417b-879d-d4c6583c4c6c'
    },
    {
        'type': 'detail',
        'value': [
            {'type': 'heading', 'value': 'Detail Überschrift', 'id': 'd7cbf386-58af-4ba6-bf7b-091e2304578a'},
            {
                'type': 'paragraph',
                'value': '<p data-block-key="g3l0n">Noch ein bisschen Text :)</p>',
                'id': 'f43fc4f3-f34f-4506-9a23-1ef4564de0e7'
            },
            {'type': 'image', 'value': 4, 'id': '45993490-7907-4d11-ac90-35bbe87f81ee'}
        ],
        'id': '6e18b320-7cfa-4255-9226-35b52b93aaac'
    }
]

# Migrate Posts

In [13]:
def is_tag(text):
    return text.startswith("{%") and text.endswith("%}")


def tag_to_block(tag):
    tag_name, tag_id = tag.strip("{%").strip("%}").split()
    tag_id = int(tag_id)
    if tag_name == "gallery":
        image_blocks = []
        for image in Gallery.objects.get(pk=tag_id).images.all():
            image_blocks.append({"type": "item", "value": image.pk, "id": str(uuid4())})
        return {"type": tag_name, "value": image_blocks}
    return {"type": tag_name, "value": tag_id}


def content_to_streamfield(content):
    blocks = overview = []
    just_tag = re.compile(r"({% \w+ \d+ %})")
    for part in just_tag.split(content):
        if len(part) == 0:
            continue
        if is_tag(part):
            blocks.append(tag_to_block(part))
        else:
            blocks.append({"type": "paragraph", "value": part})
    return [{"type": "overview", "value": overview}]


def build_post_from_legacy(legacy):
    # post = Post(title=legacy["title"], visible_date=legacy["visible_date"])
    kwargs = {
        k: v
        for k, v in legacy.items()
        if k not in ["content", "author_id", "blog_id", "id"]
    }
    kwargs["content_type"] = ContentType.objects.get(app_label="cast", model="post")
    # print(kwargs)
    post = Post(**kwargs)
    raw_body = content_to_streamfield(legacy["content"])
    post.body = json.dumps(raw_body)
    return post

In [9]:
legacy_post_lookup = {p["id"]: p for p in converter.legacy.posts}

In [13]:
# for lpost in legacy_post_lookup.values():
#     if "video" in lpost["content"] and lpost["blog_id"] == 1:
#         print(lpost)

In [14]:
blog = Page.objects.get(slug="ephes_blog")
legacy_post_lookup = {p["id"]: p for p in converter.legacy.posts}
# for p_id in (332, 333):
for p_id in (9,):
    l_post = legacy_post_lookup[p_id]
    post = build_post_from_legacy(l_post)
    post = blog.add_child(instance=post)

In [18]:
[l_post] = [p for p in converter.legacy.posts if p["id"] == 333]

In [19]:
post = build_post_from_legacy(l_post)

{
    'created': datetime.datetime(2022, 11, 2, 5, 32, 58, 340726, tzinfo=datetime.timezone.utc),
    'modified': datetime.datetime(2022, 11, 7, 13, 4, 8, 395874, tzinfo=datetime.timezone.utc),
    'title': 'Weeknotes 2022-11-07',
    'pub_date': datetime.datetime(2022, 11, 7, 13, 4, 8, 393468, tzinfo=datetime.timezone.utc),
    'visible_date': datetime.datetime(2022, 11, 7, 0, 0, tzinfo=datetime.timezone.utc),
    'slug': 'weeknotes-2022-11-07',
    'podcast_audio_id': None,
    'uuid': UUID('d402a42d-a599-4dad-b212-2f56b15b1ab5'),
    'keywords': '',
    'explicit': 1,
    'block': False,
    'comments_enabled': True,
    'content_type': <ContentType: cast | post>
}

In [20]:
print(post.body.raw_data)

[{'type': 'overview', 'value': [{'type': 'paragraph', 'value': '<p>Short work week. Gave me time to indulge in 
heavy yak shaving. At first, I tried to just make&nbsp;<a 
href="https://github.com/ephes/django-cast">django-cast</a>&nbsp;(the develop branch) work for newer <a 
href="https://www.djangoproject.com/">Django</a> and <a href="https://wagtail.org/">Wagtail</a> versions. Then I 
realized I had to fix&nbsp;<a href="http://github.com/ephes/wagtail_srcset/issues/2">wagtail-srcset</a>&nbsp;first 
and replaced&nbsp;<a href="https://python-poetry.org/">poetry</a>&nbsp;with&nbsp;<a 
href="https://flit.pypa.io/en/stable/">flit</a>&nbsp;while I&#39;m at it. Then I got lost fixing some stuff 
in&nbsp;<a href="https://github.com/ephes/kptncook/issues/18">kptncook</a>&nbsp;before I remembered that I have to 
also update&nbsp;<a href="https://github.com/ephes/django-indieweb">django-indieweb</a>&nbsp;and&nbsp;<a 
href="https://github.com/ephes/django_fileresponse/tree/main/">django-fileresponse</a>&nbsp;to be able to finally 
fix&nbsp;<a href="https://github.com/ephes/django-cast">django-cast</a>&nbsp;and make it deployable on my staging 
system using the latest and greatest Django and Wagtail versions. Oh and then I fixed some bootstrap stuff in the 
commands.py of my <a href="https://github.com/ephes/django-template/issues/7">Django startproject template</a>. 
Maybe I&#39;ll manage to merge the develop branch of <a 
href="https://github.com/ephes/django-cast">django-cast</a>&nbsp;back into main soon after develop started to 
diverge back in 2019 iirc&nbsp;because I wanted to use Wagtail as a CMS. Looking forward to meet some hairy yaks on
the way.&nbsp;<br />\r\n&nbsp;</p>\r\n\r\n<h3>Articles</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://github.com/simonw/simonwillisonblog/issues/290">Get a Mastodon instance running #290</a>&nbsp;| This 
yak is looking especially attractive atm</li>\r\n\t<li><a 
href="https://aeracode.org/2022/11/03/static-dynamic-in-memory-sqlite/">Static-Dynamic Content With In-Memory 
SQLite</a>&nbsp;| Hmm, cool - but I need to shave my postgres 15 yak first, sorry</li>\r\n\t<li><a 
href="https://fasterthanli.me/articles/becoming-fasterthanlime-full-time">Becoming fasterthanlime 
full-time</a></li>\r\n</ul>\r\n\r\n<h3><br />\r\nTwitter</h3>\r\n\r\n<ul>\r\n\t<li>No twitter this week, just 
yaks</li>\r\n</ul>\r\n&nbsp;\r\n\r\n<h3>Weeknotes</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://luisnatera.com/posts/2022/10/2022-10-31-review/">Week in Review 2022-10-31</a>&nbsp;| 
Luis</li>\r\n\t<li><a href="https://shezi.de/weeklogs/log-2022-10-30.html">Weeklog for Week 43: October 24 to 
October 30</a>&nbsp;| Johannes</li>\r\n</ul>\r\n&nbsp;\r\n\r\n<h3>Software</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://github.com/bigskysoftware/htmx/releases/tag/v1.8.4">htmx v1.8.4 release</a></li>\r\n\t<li><a 
href="https://docs.djangoproject.com/en/4.1/releases/4.1.3/">Django 4.1.3 release</a>&nbsp;| Support for Python 
3.11 + some bugfixes</li>\r\n\t<li><a 
href="https://adamj.eu/tech/2022/10/26/django-upgrade-mega-release/">django-upgrade Mega Release 
1.11.0</a></li>\r\n\t<li><a href="https://web.dev/patterns/components/">Component patterns A collection of cross 
browser UI components for use in accelerating or inspiring your own design 
systems.</a></li>\r\n</ul>\r\n\r\n<h3><br />\r\nPodcasts</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://www1.wdr.de/mediathek/audio/wdr5/wdr5-das-philosophische-radio/audio-was-sollte-ihrer-meinung-nach-de
n-kapitalismus-abloesen-100.html">Was sollte Ihrer Meinung nach den Kapitalismus abl&ouml;sen?</a>&nbsp;(<a 
href="https://www1.wdr.de/radio/wdr5/sendungen/philosophisches-radio/index.html">Das philosophische 
Radio</a>)</li>\r\n\t<li><a 
href="https://capitalisnt.simplecast.com/episodes/a-different-story-of-inflation-with-john-cochrane">A Different 
Story Of Inflation With John Cochrane</a>&nbsp;(<a href="https://capitalisnt.simplecast.com">Capitalisn&#39;t</a>) 
| Doh - bonds and cash aren&#39;t the 

In [21]:
post = blog.add_child(instance=post)

In [56]:
def is_tag(text):
    return text.startswith("{%") and text.endswith("%}")


def tag_to_block(tag):
    tag_name, tag_id = tag.strip("{%").strip("%}").split()
    tag_id = int(tag_id)
    return {"type": tag_name, "value": tag_id}


def content_to_streamfield(content):
    blocks = overview = []
    just_tag = re.compile(r"({% \w+ \d+ %})")
    for part in just_tag.split(content):
        if len(part) == 0:
            continue
        if is_tag(part):
            blocks.append(tag_to_block(part))
        else:
            blocks.append({"type": "paragraph", "value": part})
    return [{"type": "overview", "value": overview}]

In [57]:
content_to_streamfield(l_post["content"])

[{'type': 'overview',
  'value': [{'type': 'paragraph',
    'value': '<p>Short work week. Gave me time to indulge in heavy yak shaving. At first, I tried to just make&nbsp;<a href="https://github.com/ephes/django-cast">django-cast</a>&nbsp;(the develop branch) work for newer <a href="https://www.djangoproject.com/">Django</a> and <a href="https://wagtail.org/">Wagtail</a> versions. Then I realized I had to fix&nbsp;<a href="http://github.com/ephes/wagtail_srcset/issues/2">wagtail-srcset</a>&nbsp;first and replaced&nbsp;<a href="https://python-poetry.org/">poetry</a>&nbsp;with&nbsp;<a href="https://flit.pypa.io/en/stable/">flit</a>&nbsp;while I&#39;m at it. Then I got lost fixing some stuff in&nbsp;<a href="https://github.com/ephes/kptncook/issues/18">kptncook</a>&nbsp;before I remembered that I have to also update&nbsp;<a href="https://github.com/ephes/django-indieweb">django-indieweb</a>&nbsp;and&nbsp;<a href="https://github.com/ephes/django_fileresponse/tree/main/">django-filerespons

In [45]:
l_post["content"][-20:]

'3>\r\n{% image 1281 %}'

In [29]:
post = blog.add_child(instance=post)

In [14]:
post.visible_date

datetime.datetime(2022, 11, 7, 0, 0, tzinfo=datetime.timezone.utc)

In [17]:
post.delete()

# Get Templatetags from Content

In [39]:
just_tag = re.compile(r"({% \w+ \d+ %})")
just_tag.split("foo {% asdf 34 %} bar")

['foo ', '{% asdf 34 %}', ' bar']

In [38]:
#split_tags.split(legacy["content"])

In [44]:
parse_tag = re.compile(r"{% (\w+) (\d+) %}")
[(tag_name, tag_pk)] = parse_tag.findall("{% asdf 34 %}")
print(tag_name, tag_pk)

asdf 34

In [42]:
parse_tag.findall("asdf")

[]

In [21]:
tags_regex.findall(legacy["content"])

[('image', '1281')]

In [51]:
tag_name, tag_id = "{% image 1281 %}".strip("{%").strip("%}").split()
tag_id = int(tag_id)
print(tag_name, tag_id)

image 1281

In [20]:
Page.objects.last().pk

6

In [30]:
print(Page.objects.get(slug="gallerie-test").post.body.raw_data)

[{'type': 'overview', 'value': [{'type': 'heading', 'value': 'asdf', 'id': 'a5b3c0cc-889b-4b57-bd81-3a4bed41528b'},
{'type': 'gallery', 'value': [{'type': 'item', 'value': 1281, 'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'}, 
{'type': 'item', 'value': 1280, 'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}], 'id': 
'2a30b103-f138-488d-aa73-713b1de7d5ef'}], 'id': '56fa286a-3d6e-46c1-8000-b29f0cc9e847'}]

In [44]:
{
    "type": "gallery",
    "value": [
        {"type": "item", "value": 1281, "id": "88c02e62-77e9-44f2-a84e-c3d84453cb13"},
        {"type": "item", "value": 1280, "id": "eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af"},
    ],
}

{'type': 'gallery',
 'value': [{'type': 'item',
   'value': 1281,
   'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'},
  {'type': 'item',
   'value': 1280,
   'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}]}

In [25]:
Page.objects.all()

<PageQuerySet [<Page: Root>, <Page: Welcome to your new Wagtail site!>, <Page: Claas sagt Hallo>, <Page: Ephes Blog>, <Page: Gallerie-Test>, <Page: sadf>]>

In [46]:
post.body.raw_data[0]["value"][-1]

{'type': 'gallery',
 'value': [{'type': 'item',
   'value': 1281,
   'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'},
  {'type': 'item',
   'value': 1280,
   'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}]}

In [49]:
post.body.raw_data[0]["value"][-1] = {
    "type": "gallery",
    "value": [
        {"type": "item", "value": 1281, "id": "88c02e62-77e9-44f2-a84e-c3d84453cb13"},
        {"type": "item", "value": 1280, "id": "eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af"},
    ],
}

In [51]:
Post.objects.all()

<PageQuerySet [<Post: Gallerie-Test>]>

In [56]:
Page.objects.last()

<Page: sadf>

In [18]:
{
    "type": "gallery",
    "value": [
        {"type": "item", "value": 1281},
        {"type": "item", "value": 1280},
    ],
}

{'type': 'gallery',
 'value': [{'type': 'item', 'value': 1281}, {'type': 'item', 'value': 1280}]}

In [11]:
{
    "type": "gallery",
    "value": [
        {"type": "item", "value": 1281, "id": "88c02e62-77e9-44f2-a84e-c3d84453cb13"},
        {"type": "item", "value": 1280, "id": "eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af"},
    ],
}

{'type': 'gallery',
 'value': [{'type': 'item',
   'value': 1281,
   'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'},
  {'type': 'item',
   'value': 1280,
   'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}]}

In [23]:
from uuid import uuid4

In [26]:
str(uuid4())

'96bbeccf-1981-4c90-aa53-ddc703c90c8e'